## Image Segmentation

This model uses a VGG-16 U-Net Convolutional Network to segment abnormalities from mammographies of breasts

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16

In [ ]:
BATCH_SIZE = # TODO
NUM_EPOCHS = 32

In [1]:
def intersection_over_union(y_true, y_pred):
    
    intersection = (y_true * y_pred).sum()
    union = y_true.sum() + y_pred.sum() - intersection
    x = (intersection + 1e-15) / (union + 1e-15)
    x = x.astype(np.float32)
    
    return x

In [ ]:
#input is output from previous layer
#num_filters is number of feature channels in the convolutional layer
#skip_features are the appropriate size feature maps from the pre-trained VGG16 encoder.
def convolutional_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
 
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
 
    return x

In [ ]:
def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = convolutional_block(x, num_filters)
    return x

In [ ]:
#input shape is (height,   width,   channels)
def build_vgg16_unet(input_shape):
    inputs = Input(input_shape)

In [ ]:
vgg16 = VGG16(include_top=False, weights="imagenet", input_tensor=inputs)
#include_top = False means we don't want the last layer of the VGG16 model
#weights = "imagenet" means we want the weights of the VGG16 model trained on imagenet
#input_tensor = inputs means we want the input of the VGG16 model to be the input of our model

In [4]:
def build_vgg16_unet(input_shape):
    """ Input """
    inputs = Input(input_shape)
 
    """ Pre-trained VGG16 Model """
    vgg16 = VGG16(include_top=False, weights="imagenet", input_tensor=inputs)
 
    """ Encoder """
    s1 = vgg16.get_layer("block1_conv2").output         
    s2 = vgg16.get_layer("block2_conv2").output         
    s3 = vgg16.get_layer("block3_conv3").output         
    s4 = vgg16.get_layer("block4_conv3").output         
 
    """ Bridge """
    b1 = vgg16.get_layer("block5_conv3").output         
 
    """ Decoder """
    d1 = decoder_block(b1, s4, 512)                    
    d2 = decoder_block(d1, s3, 256)                     
    d3 = decoder_block(d2, s2, 128)                     
    d4 = decoder_block(d3, s1, 64)                      
 
    """ Output """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
 
    model = Model(inputs, outputs, name="VGG16_U-Net")
    return model
 
if __name__ == "__main__":
    input_shape = (512, 512, 3)
    model = build_vgg16_unet(input_shape)
    model.summary()

58900480/58889256 [==============================] - 16s 0us/step


NameError: name 'decoder_block' is not defined

In [ ]:
train_ds = self.makeTFDataset(
    shuffle=True,
    augment=True,
    x_paths_list=train_x,
    y_paths_list=train_y,
    batch_size=BATCH_SIZE,
)

In [ ]:
# Calculate train and test steps per epoch.
train_steps = len(train_x) // BATCH_SIZE
test_steps = len(test_x) // BATCH_SIZE

history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=NUM_EPOCHS,
    steps_per_epoch=train_steps,
    validation_steps=test_steps,
    callbacks=callbacks,
    verbose=1,
)